# Monte Carlo World Cup Simulation

### Elo Data
###### Source date: 05/22/2018 
https://www.eloratings.net/

In [1]:
participant_elos = {'Australia':1714, 
                    'Iran':1796,
                    'Japan':1692, 
                    'Saudi Arabia': 1600,
                    'South Korea':1745, 
                    'Egypt':1646,
                    'Morocco':1711,
                    'Nigeria':1699,
                    'Senegal':1747,
                    'Tunisia':1649,
                    'Costa Rica':1745,
                    'Mexico':1859,
                    'Panama':1669,
                    'Argentina': 1985,
                    'Brazil':2131,
                    'Colombia':1935,
                    'Peru':1906,
                    'Uruguay':1891,
                    'Belgium':1931,
                    'Croatia':1853,
                    'Denmark':1843,
                    'England':1941,
                    'France':1984,
                    'Germany':2092,
                    'Iceland':1787,
                    'Poland':1831,
                    'Portugal':1975,
                    'Russia':1685,       # Host Country
                    'Serbia':1770,
                    'Spain':2049,
                    'Sweden':1796,
                    'Switzerland':1879              
                   }

#### Groups

In [2]:
groups = {'Group_A': ['Russia', 'Saudi Arabia', 'Egypt', 'Uruguay'],
          'Group_B': ['Portugal', 'Spain', 'Morocco', 'Iran'],
          'Group_C': ['France', 'Australia', 'Peru', 'Denmark'],
          'Group_D': ['Argentina', 'Iceland', 'Croatia', 'Nigeria'],
          'Group_E': ['Brazil', 'Switzerland', 'Costa Rica', 'Serbia'],
          'Group_F': ['Germany', 'Mexico', 'Sweden', 'South Korea'],
          'Group_G': ['Belgium', 'Panama', 'Tunisia', 'England'],
          'Group_H': ['Poland', 'Senegal', 'Colombia', 'Japan']       
        }

#### Helper Functions

In [10]:
import pandas as pd

def _create_score_pandas_structure(participant_elos):
    columns = ['Second_Group_Stage', 'Winner_Group_Stage', 'Quater-Finalist', 'Semi-Finalist', 'Finalist', 'Winner']
    rows = list(participant_elos.keys())

    df = pd.DataFrame(rows)
    for column in columns:
        df[column] = 0

    df = df.set_index(0)
    df.index.names = ['Country']
    return df

def _binomial_trial(success_rate):
    if success_rate > random.random():
        return True
    return False

def _win_probability(team, opponent, elo=participant_elos):
    team_elo = elo[team]
    opponent_elo = elo[opponent]
    return (1 / (10 ** ( - ( team_elo-opponent_elo) / 400) + 1 ) )

def _stochastic_game_winner(team, opponent, elo=participant_elos):
    team_win_rate = _win_probability(team, opponent, elo)
    if _binomial_trial(team_win_rate):
        return team
    
    return opponent

#### Win probability formula
$$W_e = \frac{1}{10^{\frac{-d_r}{400}} + 1}$$


In [11]:
from itertools import combinations
import random
from random import shuffle

def single_group_run(group_name):
    group_members = groups[group_name]
    shuffle(group_members)
    final_scores = {k:0 for k in group_members}
    for game in combinations(group_members, 2):
        win_prob = _win_probability(game[0], game[1], participant_elos)
        if _binomial_trial(win_prob):
            final_scores[game[0]] += 1
        else:
            final_scores[game[1]] += 1
            
    rank = sorted(final_scores, key=lambda k: final_scores[k], reverse=True)
    return rank[0], rank[1]

##OBSOLETE
def calculate_group_outcome_probabilities(group_name, number_of_runs):
    group_members = groups[group_name]
    first_place_count = {k:0 for k in group_members}
    second_place_count = {k:0 for k in group_members}
    for run in range(number_of_runs):
        first, second = single_group_run(group_name)
        first_place_count[first] += 1
        second_place_count[second] += 1
        
    first_place_normalized = {key:float(value)/sum(first_place_count.values()) for (key,value) in first_place_count.items()}
    second_place_normalized = {key:float(value)/sum(second_place_count.values()) for (key,value) in second_place_count.items()}
        
    return first_place_normalized, second_place_normalized

###### 8ths: Winner C vs Second D | Winner A vs Second B | Winner B vs Second A | Winner D vs Second C | Winner E vs Second F | Winner G vs Second E | Winner F vs Second E | Winner H vs Second G

###### quarter-finals: 

![image.png](img/match_paths.png)


In [16]:
def world_cup_single_run():
    a_first, a_second = single_group_run('Group_A')
    b_first, b_second = single_group_run('Group_B')
    c_first, c_second = single_group_run('Group_C')
    d_first, d_second = single_group_run('Group_D')
    e_first, e_second = single_group_run('Group_E')
    f_first, f_second = single_group_run('Group_F')
    g_first, g_second = single_group_run('Group_G')
    h_first, h_second = single_group_run('Group_H')
    
    sixteen_1 = _stochastic_game_winner(a_first, b_second)
    sixteen_2 = _stochastic_game_winner(c_first, d_second)
    sixteen_3 = _stochastic_game_winner(e_first, f_second)
    sixteen_4 = _stochastic_game_winner(g_first, h_second)
    sixteen_5 = _stochastic_game_winner(b_first, a_second)
    sixteen_6 = _stochastic_game_winner(d_first, c_second)
    sixteen_7 = _stochastic_game_winner(f_first, e_second)
    sixteen_8 = _stochastic_game_winner(h_first, g_second)
    
    quarter_final_1 = _stochastic_game_winner(sixteen_1, sixteen_2)
    quarter_final_2 = _stochastic_game_winner(sixteen_3, sixteen_4)
    quarter_final_3 = _stochastic_game_winner(sixteen_5, sixteen_6)
    quarter_final_4 = _stochastic_game_winner(sixteen_7, sixteen_8)
    
    semi_final_1 = _stochastic_game_winner(quarter_final_1, quarter_final_2)
    semi_final_2 = _stochastic_game_winner(quarter_final_3, quarter_final_4)
    
    winner = _stochastic_game_winner(semi_final_1, semi_final_2)
    
    results_pandas = _create_score_pandas_structure(participant_elos)
    
    detailed_results = {'Second_Group_Stage': [a_second, b_second, c_second, d_second, e_second, f_second, g_second, h_second],
                        'Winner_Group_Stage': [a_first, b_first, c_first, d_first, e_first, f_first, g_first, h_first],
                        'Quater-Finalist': [sixteen_1, sixteen_2, sixteen_3, sixteen_4, sixteen_5, sixteen_6, sixteen_7, sixteen_8],
                        'Semi-Finalist': [quarter_final_1, quarter_final_2, quarter_final_3, quarter_final_4],
                        'Finalist': [semi_final_1, semi_final_2],
                        'Winner': [winner]
                       }
    
    for stage in detailed_results:
        for country in detailed_results[stage]:
            results_pandas.loc[country,stage] = 1
    
    return results_pandas

def calculate_world_cup_outcome_probabilities(number_of_runs):
    countries = participant_elos.keys()
    wins_total_pandas = _create_score_pandas_structure(participant_elos)
    
    for run in range(number_of_runs):
        wins_total_pandas = wins_total_pandas.add(world_cup_single_run())
    
    # Normalize
    wins_total_pandas['Second_Group_Stage'] = wins_total_pandas['Second_Group_Stage']/number_of_runs
    wins_total_pandas['Winner_Group_Stage'] = wins_total_pandas['Winner_Group_Stage']/number_of_runs
    wins_total_pandas['Quater-Finalist'] = wins_total_pandas['Quater-Finalist']/number_of_runs
    wins_total_pandas['Semi-Finalist'] = wins_total_pandas['Semi-Finalist']/number_of_runs
    wins_total_pandas['Finalist'] = wins_total_pandas['Finalist']/number_of_runs
    wins_total_pandas['Winner'] = wins_total_pandas['Winner']/number_of_runs
    
    wins_total_pandas=wins_total_pandas.sort_values(by="Winner", ascending=False)
    
    return wins_total_pandas
    
calculate_world_cup_outcome_probabilities(10000)
    

,Second_Group_Stage,Winner_Group_Stage,Quater-Finalist,Semi-Finalist,Finalist,Winner
Country,,,,,,
Brazil,0.1927,0.7408,0.7026,0.5546,0.4023,0.2676
Germany,0.2073,0.6995,0.6420,0.4817,0.3234,0.1951
Spain,0.3102,0.5395,0.7075,0.4727,0.2663,0.1468
Portugal,0.3984,0.3341,0.5503,0.3217,0.1427,0.0648
Argentina,0.2652,0.5639,0.5052,0.2752,0.1317,0.0627
France,0.2967,0.4851,0.5074,0.2841,0.1319,0.0601
England,0.3609,0.4609,0.5253,0.2057,0.1026,0.0394
Belgium,0.3767,0.4339,0.5044,0.1928,0.0913,0.0364
Colombia,0.2792,0.5220,0.4549,0.1794,0.0825,0.0309
